# Polarizability by linear response

We compute the polarizability of a Helium atom. The polarizability
is defined as the change in dipole moment
$$
μ = ∫ r ρ(r) dr
$$
with respect to a small uniform electric field $E = -x$.

We compute this in two ways: first by finite differences (applying a
finite electric field), then by linear response. Note that DFTK is
not really adapted to isolated atoms because it uses periodic
boundary conditions. Nevertheless we can simply embed the Helium
atom in a large enough box (although this is computationally wasteful).

As in other tests, this is not fully converged, convergence
parameters were simply selected for fast execution on CI,

In [1]:
using DFTK
using LinearAlgebra
using PseudoPotentialData

a = 10.
lattice = a * I(3)  # cube of $a$ bohrs
pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.largecore.gth")
# Helium at the center of the box
atoms     = [ElementPsp(:He, pseudopotentials)]
positions = [[1/2, 1/2, 1/2]]


kgrid = [1, 1, 1]  # no k-point sampling for an isolated system
Ecut = 30
tol = 1e-8

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    rr = [(r[1] - a/2) for r in r_vectors_cart(basis)]
    sum(rr .* ρ) * basis.dvol
end;

## Using finite differences
We first compute the polarizability by finite differences.
First compute the dipole moment at rest:

In [2]:
model  = model_DFT(lattice, atoms, positions;
                   functionals=LDA(), symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis; tol)
μref   = dipole(basis, scfres.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770426244253                   -0.53    9.0    173ms
  2   -2.771688158554       -2.90       -1.32    1.0    102ms
  3   -2.771714431968       -4.58       -2.53    1.0   94.0ms
  4   -2.771714670188       -6.62       -3.38    1.0   98.2ms
  5   -2.771714712279       -7.38       -3.80    2.0    114ms
  6   -2.771714715128       -8.55       -4.76    1.0    103ms
  7   -2.771714715243       -9.94       -5.11    2.0    160ms
  8   -2.771714715250      -11.15       -6.04    1.0    413ms
  9   -2.771714715250      -13.00       -6.97    1.0    125ms
 10   -2.771714715250   +  -13.80       -7.30    2.0    152ms
 11   -2.771714715250   +  -14.18       -7.90    1.0    104ms
 12   -2.771714715250      -13.85       -8.67    1.0    108ms


-0.00013457292279658894

Then in a small uniform field:

In [3]:
ε = .01
model_ε = model_DFT(lattice, atoms, positions;
                    functionals=LDA(),
                    extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                    symmetries=false)
basis_ε = PlaneWaveBasis(model_ε; Ecut, kgrid)
res_ε   = self_consistent_field(basis_ε; tol)
με = dipole(basis_ε, res_ε.ρ)

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770461135513                   -0.52    9.0    158ms
  2   -2.771777202926       -2.88       -1.32    1.0    103ms
  3   -2.771801652422       -4.61       -2.44    1.0   96.4ms
  4   -2.771801996712       -6.46       -3.14    1.0   96.4ms
  5   -2.771802073574       -7.11       -3.97    2.0    115ms
  6   -2.771802074233       -9.18       -4.29    1.0    504ms
  7   -2.771802074468       -9.63       -5.25    1.0    121ms
  8   -2.771802074476      -11.12       -5.62    2.0    157ms
  9   -2.771802074476      -12.39       -6.23    1.0    118ms
 10   -2.771802074476      -13.55       -6.42    1.0    104ms
 11   -2.771802074476   +  -15.35       -7.12    1.0    106ms
 12   -2.771802074476      -14.24       -7.39    2.0    121ms
 13   -2.771802074476   +  -14.45       -7.33    1.0    372ms
 14   -2.771802074476   +  -14.45       -7.72    1.0    139ms
 15   -2.

0.017612221262515094

In [4]:
polarizability = (με - μref) / ε

println("Reference dipole:  $μref")
println("Displaced dipole:  $με")
println("Polarizability :   $polarizability")

Reference dipole:  -0.00013457292279658894
Displaced dipole:  0.017612221262515094
Polarizability :   1.7746794185311683


The result on more converged grids is very close to published results.
For example [DOI 10.1039/C8CP03569E](https://doi.org/10.1039/C8CP03569E)
quotes **1.65** with LSDA and **1.38** with CCSD(T).

## Using linear response

Now we use linear response (also known as density-functional perturbation theory)
to compute this analytically; we refer to standard
textbooks for the formalism. In the following, $χ_0$ is the
independent-particle polarizability, and $K$ the
Hartree-exchange-correlation kernel. We denote with $δV_{\rm ext}$ an external
perturbing potential (like in this case the uniform electric field).

In [5]:
# `δVext` is the potential from a uniform field interacting with the dielectric dipole
# of the density.
δVext = [-(r[1] - a/2) for r in r_vectors_cart(basis)]
δVext = cat(δVext; dims=4)

54×54×54×1 Array{Float64, 4}:
[:, :, 1, 1] =
  5.0       5.0       5.0       5.0      …   5.0       5.0       5.0
  4.81481   4.81481   4.81481   4.81481      4.81481   4.81481   4.81481
  4.62963   4.62963   4.62963   4.62963      4.62963   4.62963   4.62963
  4.44444   4.44444   4.44444   4.44444      4.44444   4.44444   4.44444
  4.25926   4.25926   4.25926   4.25926      4.25926   4.25926   4.25926
  4.07407   4.07407   4.07407   4.07407  …   4.07407   4.07407   4.07407
  3.88889   3.88889   3.88889   3.88889      3.88889   3.88889   3.88889
  3.7037    3.7037    3.7037    3.7037       3.7037    3.7037    3.7037
  3.51852   3.51852   3.51852   3.51852      3.51852   3.51852   3.51852
  3.33333   3.33333   3.33333   3.33333      3.33333   3.33333   3.33333
  ⋮                                      ⋱                      
 -3.33333  -3.33333  -3.33333  -3.33333  …  -3.33333  -3.33333  -3.33333
 -3.51852  -3.51852  -3.51852  -3.51852     -3.51852  -3.51852  -3.51852
 -3.7037   -3.7037 

Then:
$$
δρ = χ_0 δV = χ_0 (δV_{\rm ext} + K δρ),
$$
which implies
$$
δρ = (1-χ_0 K)^{-1} χ_0 δV_{\rm ext}.
$$
From this we identify the polarizability operator to be $χ = (1-χ_0 K)^{-1} χ_0$.
Numerically, we apply $χ$ to $δV = -x$ by solving a linear equation
(the Dyson equation) iteratively.

First we do this using the `DFTK.solve_ΩplusK_split`
function provided by DFTK,
which uses an adaptive Krylov subspace algorithm [^HS2025]:

[^HS2025]:
    M. F. Herbst and B. Sun.
    *Efficient Krylov methods for linear response in plane-wave electronic structure calculations.*
    [arXiv 2505.02319](http://arxiv.org/abs/2505.02319)

In [6]:
# Multiply δVext times the Bloch waves, then solve the Dyson equation:
δVψ = DFTK.multiply_ψ_by_blochwave(scfres.basis, scfres.ψ, δVext)
res = DFTK.solve_ΩplusK_split(scfres, -δVψ; verbose=true)

Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0   6.15s  Non-interacting
   1        0       1       -0.60     10.0   13.0s  
   2        0       2       -2.42      7.0   561ms  
   3        0       3       -3.54      5.0   101ms  
   4        0       4       -5.33      4.0  99.4ms  
   5        0       5       -7.68      1.0  85.0ms  
   6        0       6      -10.06      1.0  85.1ms  
   7        1       1       -7.44     13.0   312ms  Restart
   8        1       2       -8.94      1.0  93.4ms  
                                      13.0   747ms  Final orbitals


(δψ = Matrix{ComplexF64}[[-0.0032784984946108305 - 0.00033818208549631437im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; -0.03432940014007089 + 0.30748607674740347im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; … ; -0.00437821059665114 + 0.04083109249811635im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im; 0.007996377217629289 - 0.08054222383221622im 0.0 + 0.0im 0.0 + 0.0im 0.0 + 0.0im]], δρ = [-3.620694415489122e-7 -2.5020151273711987e-7 … -4.880643121079275e-8 -2.502015083991232e-7; -6.2936171677247e-7 -4.793036739326571e-7 … -1.2077844459775273e-7 -4.793036690846498e-7; … ; 1.3626262380586923e-7 1.1356230827551441e-7 … 4.7119905543538244e-8 1.1356230873296842e-7; 1.3305479518535703e-7 1.3632642617958042e-7 … 5.200686131071553e-8 1.3632642873409454e-7;;; -2.502014819051293e-7 -1.740490577144345e-7 … -3.5564938042634736e-8 -1.7404905461332672e-7; -4.793036489543363e-7 -3.679732049999896e-7 … -1.1202894684076421e-7 -3.67973201869932e-7; … ; 1.1356231301702095e-7 1.1211099554509672e-7 … 1.0980684383632527e-7 1.12

From the result of `solve_ΩplusK_split` we can easily compute the polarisabilities:

In [7]:
println("Non-interacting polarizability: $(dipole(basis, res.δρ0))")
println("Interacting polarizability:     $(dipole(basis, res.δρ))")

Non-interacting polarizability: 1.9257125267059074
Interacting polarizability:     1.773654857431097


As expected, the interacting polarizability matches the finite difference
result. The non-interacting polarizability is higher.

### Manual solution of the Dyson equations
To see what goes on under the hood, we also show how to manually solve the
Dyson equation using KrylovKit:

In [8]:
using KrylovKit

# Apply $(1- χ_0 K)$
function dielectric_operator(δρ)
    δV = apply_kernel(basis, δρ; scfres.ρ)
    χ0δV = apply_χ0(scfres, δV).δρ
    δρ - χ0δV
end

# Apply $χ_0$ once to get non-interacting dipole
δρ_nointeract = apply_χ0(scfres, δVext).δρ

# Solve Dyson equation to get interacting dipole
δρ = linsolve(dielectric_operator, δρ_nointeract, verbosity=3)[1]

println("Non-interacting polarizability: $(dipole(basis, δρ_nointeract))")
println("Interacting polarizability:     $(dipole(basis, δρ))")

[ Info: GMRES linsolve starts with norm of residual = 4.19e+00
[ Info: GMRES linsolve in iteration 1; step 1: normres = 2.49e-01
[ Info: GMRES linsolve in iteration 1; step 2: normres = 3.77e-03
[ Info: GMRES linsolve in iteration 1; step 3: normres = 2.85e-04
[ Info: GMRES linsolve in iteration 1; step 4: normres = 4.69e-06
[ Info: GMRES linsolve in iteration 1; step 5: normres = 1.09e-08
[ Info: GMRES linsolve in iteration 1; step 6: normres = 6.27e-11
[ Info: GMRES linsolve in iteration 1; step 7: normres = 9.64e-09
[ Info: GMRES linsolve in iteration 2; step 1: normres = 8.76e-10
[ Info: GMRES linsolve in iteration 2; step 2: normres = 3.51e-11
┌ Info: GMRES linsolve converged at iteration 2, step 3:
│ * norm of residual = 3.71e-12
└ * number of operations = 12
Non-interacting polarizability: 1.9257125267058541
Interacting polarizability:     1.7736548578166573


We obtain the identical result to above.